In [1]:
import uuid
import time
import pickle
import sys
import gym.spaces
import itertools
import numpy as np
import random
import tensorflow                as tf
import tensorflow.contrib.layers as layers
from collections import namedtuple
import TicTacToe
from collections import Counter
import Players
from importlib import reload
import Policy_Gradient


/Users/christophermiller/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<module 'Players' from '/Users/christophermiller/Documents/GitHub/ai/TicTacToe/venv/Players.py'>

In [2]:
import multiplayer_tools
reload(Policy_Gradient)
reload(Players)




#Main code for running policy gradient

tf.reset_default_graph()

#Define the placeholders
observation_placeholder = tf.placeholder(shape = [None,2 , 3,3], dtype = tf.int32, name ='observe')
adv_n_placeholder = tf.placeholder(shape = [None], dtype = tf.float32, name ='adv_n')
action_placeholder = tf.placeholder(shape = [None], dtype = tf.int32, name ='action')
mask_placeholder = tf.placeholder(shape=[None, 9], dtype = tf.int32, name= 'mask')

#Define the model, action distribution
model = Policy_Gradient.TicTacToe_model(observation_placeholder, "policy_gradient", 9)
model_input_s = Policy_Gradient.policy_distribution(model)

#Define Loss functions *symbolically*
log_prob, entropy = Policy_Gradient.get_log_prob(model, action_placeholder, mask_placeholder)
loss, update_op = Policy_Gradient.loss_and_update_op(log_prob, entropy, adv_n_placeholder, entropy_coeff = 0)

#start a session
sess =tf.Session()
sess.run(tf.global_variables_initializer())

#Defines player, opponent
player = Players.NN_Player(model, model_input_s, sess, observation_placeholder, duplicate=False, deterministic = False)
opponent = Players.Random_Player()

#start an environment
env = TicTacToe.mnk_game()

#Set parameters for update
number_updates_per_expert_update = 5
number_expert_updates = 1000

for k in range(number_expert_updates):
    print("iteration number", k)
    
    batch_adv_n = []
    iteration_winners = Counter({0:0,1:0,2:0})
    
    tic = time.time()
    for i in range(number_updates_per_expert_update):
        #Player and Opponent play games, player follows the policy
        #Illegal moves are never played
        paths, batch_winners = multiplayer_tools.batch_rollout(player, opponent, env, max_time_steps=100)
        
        #The winners are recorded for recordkeeping
        iteration_winners += batch_winners
        
        #The data of the paths are extracted
        adv_n = Policy_Gradient.sum_of_rewards(paths)
        batch_adv_n = batch_adv_n + adv_n
        boards = np.concatenate([path['observation'] for path in paths])
        masks = np.concatenate([path['mask'] for path in paths])
        actions = np.squeeze(np.concatenate([path["action"] for path in paths])).astype(int)
        
        #The paths data are fed into the model for updating
        sess.run(update_op, feed_dict = {mask_placeholder: masks, adv_n_placeholder: adv_n, observation_placeholder: boards , action_placeholder: actions})
    
    
    #Unwind win data:
#     print(iteration_winners)
    print("mean adv", np.mean(batch_adv_n))
    print("iteration time", time.time() - tic)
#     print(paths[0])
    
    
#     expert_player = Players.Expert_Player()
#     _, expert_batch_winners = multiplayer_tools.batch_rollout(player, expert_player, env, max_time_steps=900)
#     player_loss_percentage_vs_expert = expert_batch_winners[2]*1.0/(expert_batch_winners[0] + expert_batch_winners[1] + expert_batch_winners[2])
#     print("loss percent vs expert", player_loss_percentage_vs_expert)
    opponent = Players.NN_Player(model, model_input_s, sess, observation_placeholder)
            
    


iteration number 0
mean adv 0.2670827722772277
iteration time 0.6402909755706787
duplicating session to freeze weights for evaluation...
INFO:tensorflow:Restoring parameters from ./to_duplicate.ckpt
iteration number 1
mean adv 0.222029191321499
iteration time 0.6604318618774414
duplicating session to freeze weights for evaluation...
INFO:tensorflow:Restoring parameters from ./to_duplicate.ckpt
Destroying NN_Player and Session...
iteration number 2
mean adv 0.15628962818003914
iteration time 0.7212059497833252
duplicating session to freeze weights for evaluation...
INFO:tensorflow:Restoring parameters from ./to_duplicate.ckpt
Destroying NN_Player and Session...
iteration number 3
mean adv 0.24032534381139492
iteration time 1.0068359375
duplicating session to freeze weights for evaluation...
INFO:tensorflow:Restoring parameters from ./to_duplicate.ckpt
Destroying NN_Player and Session...
iteration number 4
mean adv 0.20159045725646124
iteration time 0.8831532001495361
duplicating session

KeyboardInterrupt: 

0


/Users/christophermiller/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/Users/christophermiller/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less
  This is separate from the ipykernel package so we can avoid doing imports until
